# Chapter 11 - Training Deep Neural Networks

## The Vanishing/Exploding Gradients Problems

Unfortunately, gradients often get smaller and smaller as the algorithm
progresses down to the lower layers. As a result, the Gradient Descent update
leaves the lower layers’ connection weights virtually unchanged, and training
never converges to a good solution. We call this the vanishing gradients
problem. In some cases, the opposite can happen: the gradients can grow bigger
and bigger until layers get insanely large weight updates and the algorithm
diverges. This is the exploding gradients problem, which surfaces in recurrent
neural networks. Deep Neural Networks suffer from unstable gradients; different layers may learn at a widely different speeds.

To mitigate this problem, we can explore other activation functions over the logistic function. Furthermore, we can also explore other forms of network weights initialization.

### Glorot and He Initialization

To mitigate the problem of vanishing and exploding gradients, Glorot and Bengio, pointed out that we need the signal to flow properly in both directions of the network: forwards and backwards. For this to happen, they argue that we need the variance of the outputs of each layer to be equal the variance of its inputs, and also, we need the gradients to have equal variance before and after flowing through a layer in reverse direction.

To accomplish that, they proposed a initialization schema call Xavier initialization of Glorot initialization. Let's define two quantities $fan_{in}$ and $fan_{out}$:

- The $fan_{in}$ of a layer is the number of inputs in the layer, and the $fan_{out}$ is the number of outputs of the layer, see this [image](https://miro.medium.com/max/424/1*aIMnYrXAlawJEOWIUgKFug.jpeg). Thus, we can also define: $fan_{avg} = (fan_{in} + fan_{out}) / 2$.

The **Glorot initialization** procedure is that the initial weights of each layer must obey the following rules:

1. Normal distribution with mean 0 and variance $\sigma^2 = \frac{1}{fan_{avg}}$;
2. or a uniform distribution between -r and +r, with $r = \sqrt{\frac{3}{fan_{avg}}}$.

These initializations are recommend when using **tanh**, **logistic** or **softmax** activation functions. If you replace $fan_{avg}$ by $fan_{in}$, you got an initialization strategy proposed by Yann LeCun, called LeCun initialization. LeCun initialization is recommended when using SELU activation function, which will be discussed soon.

In summary:

| Initialization | Activation Functions | $\sigma^2 (Normal)$ |
|----------------|----------------------|--------------------|
|Glorot | None, tanh, logistic, softmax | $1/fan_{avg}$ |
|He | ReLu and variants (leaky ReLu, RReLu, PReLu) | $2/fan_{in}$ |
|LeCun | SELU | $1/fan_{in}$ |

These initialization strategies helps to minimize the problem with vanishing and exploding gradients, but they alone are not enough to solve it. Let's talk about activation functions.

### Nonsaturating Activation Functions

Choosing the right activation function for the architecture of the network can improve model performance and speed up training time. After years only using the logistic function as activation function, researches discovered another good activation function: ReLu.

ReLu is nice because its derivative is straightforward to compute, but it comes at a cost: the dying ReLus problem. This problem happens when somehow the combination of weights in a network causes the neurons output to be always negative, which implies that the output of the neurons will always be zero, and those neurons are effectively "dead".

---
A turnaround to this problem was the leaky ReLu function: $LeakyReLu(z) = \max(\alpha z, z)$, where $\alpha$ is tipically set to a low value, e.g., 0.02. This guarantees that when $z < 0$, the output and the gradient will not be zero, but at least a small value. Some variations of the LeakyRelu are:

- the Randomized LeakyReLu (RReLu), where $alpha$ is picked randomly in a given range during training and is fixed to and average during testing.
- the Parametric LeakyReLu (PReLu), where $\alpha$ is learned during training. This strategy works good only for huge datasets, in small datasets, it runs the risk of overfitting the data.

---

Another proposed alternative to ReLu is the ELU activation function. It has proven to outperforms all variants of ReLu and alleviate the vanishing gradients problem and avoids the dead neurons problem. Further, if its parameter $\alpha$ is set to 1, it is smooth everywhere, which helps speed up Gradient Descent.

The ELU is given by $\alpha(\exp(z) - 1)$, if $z < 0$, and $z$, if $z \geq 0 $. The problem with ELU is the exponential functial, which slower computation. It's a tradeoff that can be overcomed by the fact that the ELU function helps accelerate convergence, but predictions will be slower, anyway.

---
Last but not least, comes the SELU function. It is a Scaled variant of the ELU function (thus, SELU). Researches showed that SELU can impose the network to **self-normalize**: the output of each layer tend to have zero mean and unit variance, which solves vanishing/exploding gradients. As a result, SELU often outperforms all other activation functions. However, there are some restrictions to use it:

1. Input features must be standardized (mean 0, variance 1);
2. Every hidden layer's weights must be initialized with LeCun normal initialization (`kernel_initializer="lecun_normal"`);
3. The network architecture must be sequential. It cannot have skip connections;
4. Self-normalization is only guaranteed if all layers are dense.

To use it in keras, type:

```python
layer = keras.layers.Dense(10, activation="selu", kernel_initializer="lecun_normal")
```

### Batch Normalization

Although the right choice of the activation functions and weight initialization can reduce the danger of the vanishing/exploding gradients at training beginning, it does not guarantee that they won't appear later during training. 

Batch Normalization is a technique that consists in scaling and shifting the outputs of the neurons right before or after the activations.
    
    This operation simply zero-centers and normalizes each input, then scales and shifts the result using two new parameter vectors per layer: one for scaling, the other for shifting. In other words, the operation lets the model learn the optimal scale and mean of each of the layer’s inputs.
    
The procedure involves the computation of some quantities over the **current mini-batch**:

1. $\pmb{\mu_B} = \frac{1}{m_B}\sum_{i=1}^{m_B}\pmb{x}^{(i)}$ : vector of input means (it contains one mean per input).
2. $\pmb{\sigma_B^2} = \frac{1}{m_B}\sum_{i=1}^{m_B}(\pmb{x}^{(i)} - \pmb{\mu_B})^2$ : vector of input variances (it contains one variance per input).
3. $\pmb{\hat{x}}^{(i)} = \frac{\pmb{x}^{(i)} - \pmb{\mu_B}}{\sqrt{\sigma_B^2 + \epsilon}}$ : vector of zero-centered and normalized inputs for instance i.
4. $\pmb{\hat{z}}^{(i)} = \pmb{\gamma} \otimes \pmb{\hat{x}}^{(i)} + \pmb{\beta}$ : 

    - $\gamma$ is the output scale parameter vector for the layer (it contains one scale parameter per input)
    - $\otimes$ is the element-wise multiplication (each input is multiplied by its corresponding output scale parameter)
    - $\beta$ is the output shift (offset) parameter vector for the layer (it contains one offset parameter per input)

Further, Batch Normalization also acts like a regularizer, reducing the need for other regularization techniques (such as dropout).

### Gradient Clipping

Another popular technique to mitigate the exploding gradients problem is to clip
the gradients during backpropagation so that they never exceed some threshold.
This is called Gradient Clipping. This technique is most often used in
recurrent neural networks, as Batch Normalization is tricky to use in RNNs, as
we will see in Chapter 15. For other types of networks, BN is usually sufficient.
In Keras, implementing Gradient Clipping is just a matter of setting the
clipvalue or clipnorm argument when creating an optimizer, like this:

```python
optimizer = keras.optimizers.SGD(clipvalue=1.0)
model.compile(loss="mse", optimizer=optimizer)
```

The difference between clipvalue and clipnorm is that clipvalue effectively limits the gradient, for example, if you set clipvalue = 1 and the gradient is [0.8, 20], you would get [0.8, 1], changing the direction of the gradient. Meanwhile, clipnorm does not change gradient direction, it just rescales the gradient vector according to the l2-norm.

## Reusing Pretrained Layers

 Sometimes we don't have enough data to train a robust model. We can employ a technique called "transfer learning", that consists in reusing the lower layer of a neural network trained to solve another task. Generally, these lower layers have learned how to detect some high-level features that may be helpful for your classification problem. We will trully explore this technique in chapter 15.

## Faster Optimizers

So far we've seen four ways to speed up training and performance: applying a good initialization strategy for the connection weights, using a good activation function, using batch normalization, and reusing pretrained layers.

We can still speed up even further the training time and quality by using other optimizers than the usual Gradient Descent: momentum optimization, Nesterov Accelerated Gradient, AdaGrad, RMSProp, Adam, and Nadam.

### Momentum optimization

The idea behind momentum optimization is to add a momentum quantity in the update of the network weights, so that convergence is reached faster. In contrast, using vanilla gradient descent, the update rule does not take into account the previous updates, and the convergence is slower.

For comparison, the update rule of the vanilla SGD is:

$$
\pmb{\theta} \leftarrow \pmb{\theta} - \eta\nabla_{\pmb{\theta}}J(\pmb{\theta})
$$

where, $\nabla_{\pmb{\theta}}J(\pmb{\theta})$ is the gradient vector with respect to the weights (parameters) of the cost function $J(\pmb{\theta})$. On the other hand, momentum optimization add a momentum component:

$$
\pmb{m} \leftarrow \beta\pmb{m} - \eta\nabla_{\pmb{\theta}}J(\pmb{\theta})
$$

$$
\pmb{\theta} \leftarrow \pmb{\theta} + \pmb{m}
$$

where $\beta$ simulates the friction mechanism and prevent the momentum from growing too large, which must be set between 0 (high friction) and 1 (no friction). To use it in Keras, just add the `momentum` parameter to the SGD optimizer.

```python
optimizer = keras.optimizers.SGD(lr=0.001, momentum=0.9)
```

### Nesterov Accelerated Gradient

Nesterov Accelerated Gradient (NAG) is a variation to momentum optimization. The main difference is that it measures the gradient of the cost function not at the local position $\pmb{\theta}$, but slightly ahead in the direction of the momentum, at $\pmb{\theta} + \beta\pmb{m}$.

$$
\pmb{m} \leftarrow \beta\pmb{m} - \eta\nabla_{\pmb{\theta}}J(\pmb{\theta} + \beta\pmb{\theta})
$$

$$
\pmb{\theta} \leftarrow \pmb{\theta} + \pmb{m}
$$

This small tweak works because in general the momentum vector will be pointing in the right direction (i.e., toward the optimum), so it will be slightly more accurate to use the gradient measured a bit farther in that direction rather than the gradient at the original position. NAG is generally faster than regular momentum optimization. To use it, add the `nesterov=True` parameter to the SGD optimizer.

```python
optimizer = keras.optimizers.SGD(lr=0.001, momentum=0.9, nesterov=True)
```




### Adagrad

Sometimes the surface of the cost function is steep in the direction of some dimensions and others it is smooth. The problem is that in SGD, the update rule tends to prioritize the steepest dimensions, which implies that the connection weights will not be necessarily led to the global minimum at earlier iterations. 

Adagrad tries to solve this problem using the strategy of scaling down the gradient vector along the steepest dimensions. In short, this algorithm decays the learning rate, but it does so faster for steep dimensions than for dimensions with gentler slopes. This is called an adaptive learning rate. The update rule is the following:

$$
\pmb{s} \leftarrow \pmb{s} + \nabla_{\pmb{\theta}}J(\pmb{\theta}) \otimes\nabla_{\pmb{\theta}}J(\pmb{\theta})
$$

$$
\pmb{\theta} \leftarrow \pmb{\theta} - \eta\nabla_{\pmb{\theta}}J(\pmb{\theta}) \oslash\sqrt{\pmb{s} + \epsilon}
$$

where $\otimes$, and $\oslash$ is the element-wise multiplication and division, respectively. $\epsilon$ is a smoothing factor to avoid division by zero.

AdaGrad frequently performs well for simple quadratic problems, but it often stops too early when training neural networks. The learning rate gets scaled down so much that the algorithm ends up stopping entirely before reaching the global optimum. So even though Keras has an Adagrad optimizer, **you should not use it to train deep neural networks (it may be efficient for simpler tasks such as Linear Regression, though)**.

### RMSprop

As we've seen, Adagrad runs the risk of slowing down a bit too fast and never converging to the global optimum. The RMSprop algorithm fixes this by accumulating only the gradients from the most recent iterations (as opposed to all the gradients since the beginning of training). It does so by using exponential decay in the first step: 

$$
\pmb{s} \leftarrow \beta\pmb{s} + (1 - \beta)\nabla_{\pmb{\theta}}J(\pmb{\theta}) \otimes\nabla_{\pmb{\theta}}J(\pmb{\theta})
$$

$$
\pmb{\theta} \leftarrow \pmb{\theta} - \eta\nabla_{\pmb{\theta}}J(\pmb{\theta}) \oslash\sqrt{\pmb{s} + \epsilon}
$$

The decay rate $\beta$ is typically set to 0.9.

```python
optimizer = keras.optimizers.RMSprop(lr=0.001, rho=0.9)
```

### Adam and Nadam

Adam, which stands for adaptive moment estimation, combines the ideas of momentum optimization and RMSProp: just like momentum optimization, **it keeps track of an exponentially decaying average of past gradients**; and just like RMSProp, **it keeps track of an exponentially decaying average of past squared gradients**:

$$
\pmb{m} \leftarrow \beta_1\pmb{m} - (1 - \beta_1)\eta\nabla_{\pmb{\theta}}J(\pmb{\theta} + \beta\pmb{\theta})
$$

$$
\pmb{s} \leftarrow \beta_2\pmb{s} + (1 - \beta_2)\nabla_{\pmb{\theta}}J(\pmb{\theta}) \otimes\nabla_{\pmb{\theta}}J(\pmb{\theta})
$$

$$
\pmb{\hat{m}} \leftarrow \frac{\pmb{m}}{1 - \beta_1^t}
$$

$$
\pmb{\hat{s}} \leftarrow \frac{\pmb{s}}{1 - \beta_2^t}
$$

$$
\pmb{\theta} \leftarrow \pmb{\theta} + \eta\pmb{\hat{m}}\oslash\sqrt{\pmb{\hat{s}} + \epsilon}
$$

Where $t$ is the iteration number, starting at 1

The only difference is that step 1 computes an exponentially decaying average rather than an exponentially decaying sum, but these are actually equivalent except for a constant factor (the decaying average is just $1 – \beta_1$ times the decaying sum).

Steps 3 and 4 are somewhat of a technical detail: since $\pmb{m}$ and $\pmb{s}$ are initialized at 0, they will be biased toward 0 at the beginning of training, so these two steps will help boost $\pmb{m}$ and $\pmb{s}$ at the beginning of training.

```python
optimizer = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999)
```

There is also two variations of the Adam algorithm that are worth mentioning.

- Adamax: Note that Adam scales down the parameter updates by the square root of $\pmb{s}$. In short, Adam scales down the parameter updates by the $l_2$ norm of the time-decayed gradients. Adamax, replaces the $l_2$ norm with the $l_\infty$ norm (a fancy way of saying the max). In practice, this can make AdaMax more stable than Adam, but it really depends on the dataset, and in general Adam performs better.


- Nadam: Nadam optimization is Adam optimization plus the Nesterov trick (evaluate the cost function slightly ahead in the direction of the momentum), so it will often converge slightly faster than Adam. Nadam generally outperforms Adam but is sometimes outperformed by RMSProp.

## Learning Rate Scheduling

Choosing the right learning rate is a hard task, but we can do better than using a constant learning rate. By using learning rate scheduling, the algorithm is capable of adjusting the learning rate during training so that convergence is reached faster and the solution may be even better.

Some techniques are:

- Power scheduling: Reduces the learning rate based on the number of iterations, but the reduction gets more and more slowly.

- Exponential scheduling: Gradually drops the learning rate. While power scheduling reduces the learning rate more and more slowly, exponential scheduling keeps slashing it by a factor of 10 every $s$ steps.

- Piecewise scheduling: Use a constant learning rate for a number of epochs (e.g., η = 0.1 for 5 epochs), then a smaller learning rate for another number of epochs (e.g., η = 0.001 for 50 epochs), and so on.

- Performance scheduling: Measure the validation error every N steps (just like for early stopping), and reduce the learning rate by a factor of λ when the error stops dropping.

- 1cycle scheduling: Contrary to the other approaches, 1cycle (introduced in a 2018 paper by Leslie Smith) starts by increasing the initial learning rate η0, growing linearly up to η1 halfway through training. Then it decreases the learning rate linearly down to η0 again during the second half of training, finishing the last few epochs by dropping the rate down by several orders of magnitude (still linearly).

## Avoiding Overfitting Through Regularization

We already studied some regularization techniques. In chapter 10, we studied Early Stopping. Further, in this chapter we saw Batch Normalization, which is designed to solve the unstable gradient problems, but it also acts like a good regularizer. In this section, we are going to see other popular techniques such as $l_1$, $l_2$, dropout, and max-norm regularization.

### $\ell_1$ and $\ell_2$ Regularization

$\ell_1$ and $\ell_2$ norms were already explained in chapter 4. Put simply, $\ell_1$ regularization helps us creating sparse models, effectivelly shrinking little weights to zero. Meanwhile, $\ell_2$ constrains the network weights such that they can't increase infinitely and the weights are penalized for being huge. So, here, we are going to just talk about how to use them in keras. 

```python
layer = keras.layers.Dense(100, activation="elu", 
                           kernel_initializer="he_normal", kernel_regularizer=keras.regularizers.l2(0.01))
```

The `l2()` function returns a regularizer that will be called at **each step** during training to compute the regularization loss. This is then added to the final loss. As you might expect, you can just use `keras.regularizers.l1()` if you want $\ell_1$ regularization; if you want both $\ell_1$ and $\ell_2$ regularization, use `keras.regularizers.l1_l2()` (specifying both regularization factors).

### Dropout

Dropout is another powerful regularization technique. The algorithm is simple: 

at **every training step**, every neuron (including the input neurons, but always excluding the output neurons) has a probability $p$ of being temporarily "dropped out," meaning it will be entirely ignored during this training step, but it may be active during the next step (see Figure 11-9). The hyperparameter $p$ is called the dropout rate, and it is typically set between 10% and 50%: closer to 20–30% in recurrent neural nets (see Chapter 15), and closer to 40–50% in convolutional neural networks (see Chapter 14). After training, neurons don’t get dropped anymore.

The idea behind the success of this technique is that Neurons trained with dropout cannot co-adapt with their neighboring neurons; they have to be as useful as possible on their own. They also cannot rely excessively on just a few input neurons; they must pay attention to each of their input neurons. They end up being less sensitive to slight changes in the inputs. In the end, you get a more robust network that generalizes better.

The following code applies dropout regularization before every Dense layer, using a dropout rate of 0.2:

```python
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(300, activation="elu", kernel_initializer="he_normal"),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal"),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(10, activation="softmax")
])
```


*If you want to regularize a self-normalizing network based on the SELU activation function (as
discussed earlier), you should use alpha dropout: this is a variant of dropout that preserves the
mean and standard deviation of its inputs (it was introduced in the same paper as SELU, as
regular dropout would break self-normalization).*

### Monte Carlo (MC) Dropout

In 2016, a paper by Yarin Gal and Zoubin Ghahramani added a few more good reasons to use dropout: First it established a mathematical justification to use dropout. Secondly, the authors introduced a powerful technique call Monte Carlo Dropout, which consists in making several predictions of the same data using the droupout layers active, so that every prediction for each instance will be different. Then, average all the predictions. The averaged predictions are usually better, and plus that, we can get a standard deviation estimate of the prediction probabilities.

The following code is the full implementation of the technique:

```python 
y_probas = np.stack([model(X_test_scaled, training=True) for sample in range(100)])
y_proba = y_probas.mean(axis=0)
```

We just make 100 predictions over the test set, setting `training=True` to ensure that the Dropout layer is active, and stack the predictions. Since dropout is active, all the predictions will be different.
Recall that `predict()` returns a matrix with one row per instance and one column per class. Because there are
10,000 instances in the test set and 10 classes, this is a matrix of shape [10000, 10]. We stack 100 such matrices, so `y_probas` is an array of shape [100, 10000, 10]. Once we average over the first dimension `(axis=0)`, we get `y_proba`, an array of shape [10000, 10], like we would get with a single prediction.

### Max-Norm Regularization

Another regularization technique that is popular for neural networks is called max-norm regularization: for each neuron, it constrains the weights $w$ of the incoming connections such that $∥ w ∥_2 ≤ r$, where $r$ is the max-norm
hyperparameter.

Max-norm regularization does not add a regularization loss term to the overall loss function. Instead, it is typically implemented by computing $∥w∥$ after each training step and rescaling $w$ if needed ($w \leftarrow w\frac{r}{∥ w ∥_2}$).

Reducing $r$ increases the amount of regularization and helps reduce overfitting. Max-norm regularization can also help alleviate the unstable gradients problems (if you are not using Batch Normalization).

```python
keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal",
                   kernel_constraint=keras.constraints.max_norm(1.))
``` 

After **each training iteration**, the model's `fit()` method will call the object returned by `max_norm()`, passing it the layer's weights and getting rescaled weights in return, which then replace the layer's weights.

## Exercises

1. **Is it OK to initialize all the weights to the same value as long as that value is selected randomly using He initialization?**

Independently of the initialization method, initializing all the weights with the same value will disable the network from learning, since all gradients will be the same, the neurons won't be able to learn the data patterns. It is like all the layers just had one neuron. 

2. **Is it OK to initialize the bias terms to 0?**

Yes, since backpropagation will deal with the learning process, if 0 is not a good value, it would be changed when backprop comes into action.

3. **Name three advantages of the SELU activation function over ReLU.**

neurons with selu activation function do not suffer from the dead neurons problem. Also, if the parameter $\alpha$ equal 1, then SELU is continuous. And finally, given some conditions, SELU function can add the self normalization property to the network.

4. **In which cases would you want to use each of the following activation functions: SELU, leaky ReLU (and its variants), ReLU, tanh, logistic, and softmax?**

- SELU

I would use SELU under the conditions for self-normalization of the network, which would avoid the vanishing/exploding gradients problem. These conditions are: Standardized input features, layers weights initialized with LeCun normal initialization, no incompatible layer (dropout or $\ell_1$ regularization), and sequential network architecture.

- leakyRELU

I would use leakyRELU when it is not possible to use SELU, and I want to reduce latency and computation time of the gradients. Further, if I identified the dead neurons problem.

- ReLU

I would use ReLU to speed up training time, since it is linear for $z > 0$, and 0 otherwise.

- tanh

I would use tanh in classification problems where the outputs are $\{-1, 1\}$.

- logistic

I would use the logistic sigmoid function when the output of the network can be interpreted as probabilities in classification problems,

- softmax

I would use softmax activation function for multiclass classification problems, where the output of each class can be interpreted as mutual exclusive probabilities.

5. **What may happen if you set the momentum hyperparameter too close to 1 (e.g., 0.99999) when using an SGD optimizer?**

The momentum parameter $\beta$ represents the physical friction applied to the momentum of gradient of the loss function. If $\beta$ is close to zero, the friction is very high, thus the momentum is slowed. On the other hand, if $\beta$ is close to 1, there is no friction to the system, implying that the momentum can make the gradients grow very large until they explode.

6. **Name three ways you can produce a sparse model**

I can produce a sparse model using strong $\ell_1$ regularization, or get rid of the tiny weights of the network. Another option is to use the TensorFlow Model Optimization Toolkit.

7. **Does dropout slow down training? Does it slow down inference (i.e., making predictions on new instances)? What about MC Dropout?**

Yes, dropout does slow down training, in general roughly by a factor of two. In addition, optimization methods using vectorized operations with GPU, for instance, can not be applied in the presence of droupout. Also, dropout does not change inference time, you just have to be careful to deal with the scaling of the neurons weights since when trained, the number of neurons before and after each hidden layer is reduced by the probability of droupout $p$.

In the case of MC dropout, the above is also applied to training the model. But in this case, inference time can significantly increase, because several predictions must be made in order to get an average prediction of each instance.